<a href="https://colab.research.google.com/github/lro99/stock_sentiment/blob/main/SentimentTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install hf_xet

In [2]:
pip install newsapi-python

In [3]:
pip install finnhub-python

In [4]:
# apikey = '62638dc7df5e4e958183e238948a0ebf'

In [5]:
finhubkey = 'd06ltdpr01qg26s8pi6gd06ltdpr01qg26s8pi70'

In [6]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# finBERT
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
# newsapi
from newsapi import NewsApiClient
import requests
import finnhub
# timeseries
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
# deep learning
import tensorflow as tf
from tensorflow.keras import layers

In [7]:
# get list of news
news = yf.Search("S&P 500", news_count=10).news
for i in news:
  print(i['title'])

Stock market today: S&P 500, Nasdaq futures rise after landmark antitrust ruling allows Google to keep Chrome
US Stock Futures Steady as Investors Weigh Tariff Court Verdict
Stocks steady, Target CEO, US gov't & chipmakers: 3 Things
Stock market today: Nasdaq drops over 1% as Palantir, AMD drag down tech
Stocks rise, gold spikes, Trump's Fed board pick: 3 Things
Trump vs. Powell, earnings, retail sales data: 3 Things
June PPI data: How markets are moving in response
Tariff timeline, chip stock slide, inflation jitters: 3 things
Tariff threats, Bitcoin surges, inflation data: 3 things
Tariffs, copper turbulence, Fed chair showdown: 3 Things


In [8]:
# S&P500 yahoo

ticker = yf.Ticker("SPY")
historical = ticker.history(period="10y")
historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2015-09-03 00:00:00-04:00,165.442071,166.951002,164.346215,164.843567,152087800,0.0,0.0,0.0
2015-09-04 00:00:00-04:00,162.567570,163.418969,161.522278,162.348389,207081000,0.0,0.0,0.0
2015-09-08 00:00:00-04:00,165.172337,166.580102,164.523243,166.428360,116025700,0.0,0.0,0.0
2015-09-09 00:00:00-04:00,168.021580,168.148021,163.831998,164.202896,149347700,0.0,0.0,0.0
2015-09-10 00:00:00-04:00,164.009022,166.251336,163.747702,165.096466,158611100,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-08-26 00:00:00-04:00,642.200012,645.510010,641.570007,645.159973,51581600,0.0,0.0,0.0
2025-08-27 00:00:00-04:00,644.570007,647.369995,644.419983,646.630005,48341100,0.0,0.0,0.0
2025-08-28 00:00:00-04:00,647.239990,649.479980,645.340027,648.919983,61519500,0.0,0.0,0.0


In [9]:
historical['Return'] = historical['Close'].pct_change() * 100
historical['Lag1'] = historical['Return'].shift(1)
historical['Lag2'] = historical['Return'].shift(2)
historical['Lag3'] = historical['Return'].shift(3)
historical['MA5'] = historical['Close'].rolling(5).mean()
historical['MA20'] = historical['Close'].rolling(20).mean()
historical['Volatility10'] = historical['Return'].rolling(10).std()
historical['RoC10'] = historical['Close'].pct_change(periods=10)
historical.dropna(inplace=True)


X = historical[['Lag1', 'Lag2', 'Lag3', 'MA5', 'MA20', 'Volatility10', 'RoC10']]
y = historical['Return']

# Exploratory Data Analysis

In [10]:
historical.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Return,Lag1,Lag2,Lag3,MA5,MA20,Volatility10,RoC10
count,2494.000000,2494.000000,2494.000000,2494.000000,2.494000e+03,2494.000000,2494.0,2494.0,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000
mean,341.114955,342.996708,339.077990,341.189997,8.328241e+07,0.022516,0.0,0.0,0.061551,0.062596,0.062859,0.061710,340.802593,339.365601,0.923179,0.006030
std,127.922146,128.566284,127.195376,127.952184,4.160596e+07,0.181136,0.0,0.0,1.141004,1.141463,1.141386,1.142533,127.718776,126.868425,0.694452,0.032156
min,155.426454,156.926685,154.301355,155.869705,2.027000e+07,0.000000,0.0,0.0,-10.942359,-10.942359,-10.942359,-10.942359,157.797882,160.583529,0.126446,-0.232884
25%,237.058548,238.360176,235.234336,237.036552,5.750388e+07,0.000000,0.0,0.0,-0.364043,-0.363793,-0.363724,-0.363793,236.695243,236.898369,0.506488,-0.006742
50%,314.203581,316.232897,312.404139,314.166351,7.373655e+07,0.000000,0.0,0.0,0.066787,0.068336,0.068336,0.066787,313.951645,314.033528,0.757870,0.009165
75%,423.951729,426.259529,421.791879,424.277451,9.616202e+07,0.000000,0.0,0.0,0.599462,0.599617,0.599617,0.599617,423.772313,422.229701,1.140277,0.024074
max,647.469971,649.479980,645.340027,648.919983,3.922207e+08,1.966000,0.0,0.0,10.501936,10.501936,10.501936,10.501936,645.697986,640.632495,7.073329,0.187980


# Baseline

Rolling Avg

In [11]:
window = 5

historical['RollingAvg_Pred'] = historical['Return'].shift(1).rolling(window).mean()


In [12]:
true_vals = historical['Return'].dropna()
rolling_pred = historical['RollingAvg_Pred'].dropna()
aligned = true_vals.loc[rolling_pred.index]

mse = mean_squared_error(aligned, rolling_pred)
mae = mean_absolute_error(aligned, rolling_pred)
r2 = r2_score(aligned, rolling_pred)

print(f"Rolling Avg Mean Squared Error: {mse}")
print(f"Rolling Avg Mean Absolute Error: {mae}")
print(f"Rolling Avg R-squared: {r2}")

Rolling Avg Mean Squared Error: 1.5737448900806312
Rolling Avg Mean Absolute Error: 0.8197282519358131
Rolling Avg R-squared: -0.20903055183980634


Naive

In [13]:
historical['Naive_Pred'] = historical['Return'].shift(1)

In [14]:
naive_pred = historical['Naive_Pred'].dropna()
aligned = true_vals.loc[naive_pred.index]

mse = mean_squared_error(aligned, naive_pred)
mae = mean_absolute_error(aligned, naive_pred)
r2 = r2_score(aligned, naive_pred)

print(f"Naive Mean Squared Error: {mse}")
print(f"Naive Mean Absolute Error: {mae}")
print(f"Naive R-squared: {r2}")

Naive Mean Squared Error: 2.9532264149096425
Naive Mean Absolute Error: 1.0871637996479186
Naive R-squared: -1.2684416017964546


# Model with cross validation

RandomForest

In [15]:
tscv = TimeSeriesSplit(n_splits=5)

model = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 300],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5]
}


scorer = make_scorer(mean_squared_error, greater_is_better=False)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=tscv,
    scoring=scorer,
    verbose=1
)

grid_search.fit(X, y)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 300}
Best Score: -1.294007079383732


LSTM

In [ ]:
X = X.values
y = y.values

def create_sequences(X, y, time_steps=10):
  Xs, ys = [], []
  for i in range(len(X) - time_steps):
    Xs.append(X[i:(i + time_steps)])
    ys.append(y[i + time_steps])
  return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X, y, time_steps=10)

In [ ]:
def lstm_model(input_shape):
  model = tf.keras.Sequential()

  model.add(layers.LSTM(64, activation=None, input_shape=input_shape, dropout=0.2, return_sequences=False))
  model.add(layers.Dense(1))
  model.compile(optimizer='adam', loss='mse')
  return model


In [ ]:
for train_idx, val_idx in tscv.split(X_seq):
  X_train, X_val = X_seq[train_idx], X_seq[val_idx]
  y_train, y_val = y_seq[train_idx], y_seq[val_idx]

  model = lstm_model(X_train.shape[1:])
  model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

NameError: name 'tscv' is not defined

# finBERT

In [16]:
# finBERT model
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

headline = news[9]['title']
res = finbert(headline)
print(res)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'neutral', 'score': 0.8811177015304565}]


In [17]:
for i in news:
  headline = i['title']
  res = finbert(headline)
  print(headline, res)

Stock market today: S&P 500, Nasdaq futures rise after landmark antitrust ruling allows Google to keep Chrome [{'label': 'negative', 'score': 0.4463648200035095}]
US Stock Futures Steady as Investors Weigh Tariff Court Verdict [{'label': 'negative', 'score': 0.3994804918766022}]
Stocks steady, Target CEO, US gov't & chipmakers: 3 Things [{'label': 'positive', 'score': 0.7164199352264404}]
Stock market today: Nasdaq drops over 1% as Palantir, AMD drag down tech [{'label': 'negative', 'score': 0.970177173614502}]
Stocks rise, gold spikes, Trump's Fed board pick: 3 Things [{'label': 'neutral', 'score': 0.7260529398918152}]
Trump vs. Powell, earnings, retail sales data: 3 Things [{'label': 'neutral', 'score': 0.9313461780548096}]
June PPI data: How markets are moving in response [{'label': 'neutral', 'score': 0.899533748626709}]
Tariff timeline, chip stock slide, inflation jitters: 3 things [{'label': 'negative', 'score': 0.5899151563644409}]
Tariff threats, Bitcoin surges, inflation data:

In [18]:
# graphing % change
historical = historical.reset_index()
X = pd.to_datetime(historical['Date'])
y = historical['Change']

plt.plot(X, y)
plt.title('S&P 500')
plt.xlabel('Date')
plt.ylabel('Interday Change (%)')
plt.show()


KeyError: 'Change'

# Finnhub API

In [19]:
# finnhub news api. allows for historical search

finnhub_client = finnhub.Client(api_key=finhubkey)

news = finnhub_client.company_news('SPY', _from="2025-04-01", to="2025-04-01")

# for i in news:
#   print(i['headline'])
for i in news:
  sentiment = finbert(i['summary'])
  if sentiment[0]['label'] == 'neutral':
    continue
  print(i['summary'], sentiment)

Stocks ended mostly higher today, though not in any inspiring fashion - more like the way a snake clears a hurdle. [{'label': 'positive', 'score': 0.8160881996154785}]
Expectations are building for a Fed interest rate cut. [{'label': 'positive', 'score': 0.7386780977249146}]
The S&P 500 and Nasdaq indices have breached March lows, while the Dow Jones remains resilient. Click here to find out more on Market Outlook. [{'label': 'negative', 'score': 0.7389934659004211}]
Tariffs are meant to reinvigorate US manufacturing but there's concern about what they mean for supply chains and the prospect of foreign retaliation amid signs of a cooling domestic economy [{'label': 'negative', 'score': 0.951738715171814}]
The iShares S&P GSCI Commodity-Indexed Trust rose 2.6%, posting Marchâs strongest return. Click to read. [{'label': 'positive', 'score': 0.9404349327087402}]
There are many reasons to think that the blunder of erecting a high import tariff wall will cause an economic recession not o